# Introduction

julie.tsitron@parks.nyc.gov 1/8/2020

This notebook is for testing various ArcGIS API for Python functions that allow for pushing/updating GIS data to production servers. Ultimately, the script from this notebook can be used as a template for the technical implementation (i.e., pushing clean data into production ESRI-based geodatabases) of Structures, Sites and Units, CPAs, and other Agency Data Model data products.

# Imports and Connections to DBs

In [ ]:
import urllib3
from urllib.parse import quote
from urllib.parse import quote_plus

import numpy as np

import pandas as pd
import pyodbc
import sys
sys.path.append('../') ## <-- THIS IS THE PART THAT TELLS IT TO LOOK IN THE PARENT DIRECTORY
from IPM_Shared_Code.Python.geo_functions import read_geosql
from IPM_Shared_Code.Python.email_functions import get_contacts, read_template, send_email
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor, SpatialDataFrame, FeatureLayer
import json
from datetime import datetime
import arcgis
import utils
import urllib
import sqlalchemy
import os
import shapely
import pytz

In [ ]:
config = utils.get_config('config.ini')

driver = config['srv']['driver']
vpipm_server = config['srv']['vpipm']
parksgis_dev_server = config['srv']['parksgis_dev']
parksgis_prod_server = config['srv']['parksgis_prod']
data_parks_server = config['srv']['data_parks']

structuresdb = config['db']['structuresdb']

geo_key = config['keys']['geosupport_key']
geo_ip = config['keys']['geosupport_ip']

portal_dev = config['url']['portal_dev']
portal_prod = config['url']['portal_prod']
structures_dev_url = config['url']['structures_dev']
structures_prod_url = config['url']['structures_prod']
geosupport_dev_url = config['url']['geosupport_dev']
geosupport_prod_url = config['url']['geosupport_prod']

In [ ]:
con_vpipm = pyodbc.connect('Driver={' + driver + '};Server=' + vpipm_server +
                           ';Database=' + structuresdb +
                           ';Trusted_Connection=Yes;')
con_gis_dev = pyodbc.connect('Driver={SQL Server};Server=' +
                             parksgis_dev_server +
                             ';Database=;Trusted_Connection=Yes;') 
con_gis_prod = pyodbc.connect('Driver={SQL Server};Server=' +
                             parksgis_prod_server +
                             ';Database=;Trusted_Connection=Yes;') 

In [ ]:
crsr = con_vpipm.cursor()
crsr.execute("{CALL structuresdb.dbo.sp_i_tbl_overlap_exceptions}")
crsr.commit()
crsr.close()

In [ ]:
crsr = con_vpipm.cursor()
crsr.execute("{CALL structuresdb.dbo.sp_i_tbl_audit_structures}")
crsr.commit()
crsr.close()
# con_vpipm.close()

In [ ]:
crsr = con_vpipm.cursor()
sql_stmt = '''
exec [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] 
'''
crsr.execute("{CALL [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] }")
crsr.commit()
crsr.close()

In [ ]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' +
                                 vpipm_server + ';Database=' + structuresdb +
                                 ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
connection = engine.connect()

# Data

# Deal with Dates

## m/d/Y H:M:S format:

In [ ]:
# structures_dev['COMMISSIONDATE'] = pd.to_datetime(
#     structures_dev['COMMISSIONDATE'],
#     errors='coerce').dt.strftime('%m/%d/%Y %H:%M:%S')

In [ ]:
# structures_dev['COMMISSIONDATE']

In [ ]:
# structures_dev['FEATURESTATUSCHANGEDATE'] = pd.to_datetime(
#     structures_dev['FEATURESTATUSCHANGEDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [ ]:
# structures_dev['RETIREDDATE'] = pd.to_datetime(
#     structures_dev['RETIREDDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [ ]:
# structures_dev.columns.values

## Delta Table from structuresdb (on vpipm)

In [ ]:
sql_structures = '''
select * from [ParksGIS].[DPR].[STRUCTURE_EVW] 
'''
structures_latest = read_geosql(sql_structures,
                            con_gis_dev,
                            geom_raw='SHAPE',
                            geom_col='geometry')

In [ ]:
def make_tz_aware_UTC(row, column):
    return pd.to_datetime(row[column]).tz_localize('UTC').astimezone(pytz.UTC)

In [ ]:
def make_tz_aware_EST(row, column):
    return pd.to_datetime(row[column]).tz_localize('EST').astimezone(pytz.UTC)

In [ ]:
structures_latest['last_edited_date'] = structures_latest.apply(lambda row: make_tz_aware_UTC(row, 'last_edited_date'), axis=1)

In [ ]:
# SPATIAL DATASET:
con_vpipm = pyodbc.connect('Driver={' + driver + '};Server=' + vpipm_server +
                           ';Database=' + structuresdb +
                           ';Trusted_Connection=Yes;')
sql_str_deltas = '''
select * FROM [structuresdb].[dbo].[tbl_delta_structures] 
'''

struct_deltas = read_geosql(sql_str_deltas,
                            con_vpipm,
                            geom_raw='shape',
                            geom_col='geometry')

In [ ]:
## Still need this ??

struct_deltas.rename(columns={
    'objectid': 'OBJECTID',
    'parks_id': 'SYSTEM',
    'bin': 'BIN',
    'bbl': 'BBL',
    'doitt_id': 'DOITT_ID',
    'ground_elevation': 'Ground_Elevation',
    'height_roof': 'Height_Roof',
    'alteration_year': 'Alteration_Year',
    'construction_year': 'Construction_Year',
    'demolition_year': 'Demolition_Year'
},
                     inplace=True)

In [ ]:
struct_deltas['date_stamp'] =struct_deltas.apply(lambda row: make_tz_aware_UTC(row, 'date_stamp'), axis=1)

In [ ]:
last_push_api = structures_latest[
    (structures_latest['last_edited_user'] == 'NYCDPR\py_services')
    & (structures_latest['last_edited_date'] ==
       structures_latest['last_edited_date'].max())]['last_edited_date'].loc[0]
last_push_api

In [ ]:
struct_deltas[struct_deltas['date_stamp']>last_push_api]

In [ ]:
struct_deltas = struct_deltas[struct_deltas['date_stamp']>last_push_api]

In [ ]:
struct_deltas.head()

In [ ]:
# def to_wkt(row):
#     return row.wkt

# Write Delta Table to geojson file

In [ ]:
if not struct_deltas.empty:
    today = datetime.now().strftime('%Y%m%d')
    snapshot = r'C:\\Projects\\AgencyDataModel\\Develop\\Structures\\delta_snapshots/' + today
    if os.path.exists(snapshot):
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')
    else:
        os.makedirs(snapshot)
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')

    # Read geojson file to geojson object 

    with open(snapshot+'/deltas.geojson') as data:
        geojson_deltas = json.load(data)

    # Create arcgis featureSet from geojson object

    fromJSON_deltas = arcgis.features.FeatureSet.from_geojson(geojson_deltas)

    # Connect to published dataset via GIS object

    gis = GIS(url=portal_dev)

    # Connect to feature layer directly

    strct_lyr_url = structures_dev_url

    lyr_structures = FeatureLayer(strct_lyr_url)
    structures_features = lyr_structures.query()

    structures_features

    len(struct_deltas)

    # Make Edits

    update_result = lyr_structures.edit_features(updates=fromJSON_deltas.features)

    lyr_structures = FeatureLayer(strct_lyr_url)
    structures_features = lyr_structures.query()
else:
    # Connect to published dataset via GIS object
    gis = GIS(url=portal_dev)

    # Connect to feature layer directly

    strct_lyr_url = structures_dev_url

    lyr_structures = FeatureLayer(strct_lyr_url)
    structures_features = lyr_structures.query()

    structures_features
        

# Connect to feature service

In [ ]:
gis = GIS(url=portal_dev)

## geosupport table

In [ ]:
geosupport_tbl_url = geosupport_dev_url

In [ ]:
tbl_geosupport = FeatureLayer(geosupport_tbl_url)
geosupport = tbl_geosupport.query()

# Get latest BINs from structures layer

In [ ]:
structures_valid_BINs = structures_features.sdf[
    (structures_features.sdf['BIN'] != 0) &
    (~pd.isnull(structures_features.sdf['BIN']))]  #['BIN']

In [ ]:
bins = structures_valid_BINs[structures_valid_BINs['BIN'].astype(int)%100000!=0]['BIN'].astype(int).to_list()

In [ ]:
bins = [str(bins[i]) for i in range(0, len(bins))]

In [ ]:
api_key = geo_key
grc_err = ['01/F', '20', '21', '22', '23']
out_keys = [
    'AddressRangeList', 'out_bbl', 'out_TPAD_bin', 'out_TPAD_bin_status',
    'out_TPAD_conflict_flag', 'out_error_message', 'out_grc',
    'out_sanborn_boro', 'in_bin', 'out_bbl_boro'
]

In [ ]:
def funcbn(bn=None, out_keys=None, grc_err=None,api_key=None,ip=None):
    
    url = 'http://{}/geoservice/geoservice.svc/Function_BIN?BIN={}&key={}'.format(ip, bn, api_key)
    #Encode the url, but allow the characters specified in the safe argument.
    url = quote(url, safe = ':/?&=')
    
    geo_dict = {}
    obj_to_return = {}
    #Establish the connection
    http = urllib3.PoolManager()
    try:
        #Send the get request to the AWS API and retrieve the response
        response = http.request('GET', url)
        #Load the JSON that is returned keeping only the values in the display key
        raw_dict = json.loads(response.data).get('display')
        
        #Keep only the dictionary keys the function caller wants to retain
        geo_dict = {k: raw_dict[k] for k in out_keys}                
        
        if geo_dict['out_grc'].strip() in grc_err:
            #bin_dict.update(geo_dict)
            obj_to_return['bin'] = bn
            del geo_dict['AddressRangeList']
            obj_to_return.update(geo_dict)
        else:
            geo_dict2 = geo_dict.copy()
            del geo_dict2['AddressRangeList']
            for i in geo_dict['AddressRangeList']:
                i.update(geo_dict2)
            obj_to_return = geo_dict['AddressRangeList']
        
    except:       
        print('exception!')
        #Set the output equal to the input BIN so that no information is lost
        add_dict = [bin_dict]
    # returns either list or dict:
    return obj_to_return

In [ ]:
def func1b(borough=None, addressno=None, streetname=None, api_key=None, ip=None):
    url = ('http://{}/geoservice/geoservice.svc/Function_1B?Borough={}&AddressNo={}&StreetName={}&key={}'.
           format(ip, borough, addressno, streetname, api_key))
    #Encode the url, but allow the characters specified in the safe argument.
    url = quote(url, safe = ':/?&=')
    
    #print('Checking Addresses: {} {} {}'.format(borough, addressno, streetname))
    #Feed in the url with the BIN and the API Key and read the results
    http = urllib3.PoolManager()
    response = http.request('GET', url)

    #If any of these Return Codes are output then send the (includes all return codes for functions 1, 1A, 1B and 1E)
    #REFERENCE: https://nycplanning.github.io/Geosupport-UPG/appendices/appendix01/#function-1
    out_grc = ['01/E', '01/V', '01/P', '01/8', '01/A', '??/1', 
               '04', '07', '28', '29' '41', '42', '50', '56', 
               '69/B', '73', '75', '89', '90']

    #Create a tuple of the keys that need to be retained
    out_keys = ('out_zip_code', 'out_hurricane_zone', 'out_co', 'out_cd', 
                'out_sd','out_nta', 'out_ad', 'out_com_dist', 'out_fire_bat', 
                'out_fire_co', 'out_fire_co_str', 'out_fire_div', 'out_b10sc1',
                'out_police_patrol_boro', 'out_police_area', 'out_police_pct', 
                'out_san_sched', 'out_san_dist_section', 'out_san_recycle', 'out_san_reg', 'out_san_org_pick_up',
                'out_usps_city_name', 'out_preferred_lgc', 'out_sos_ind', 'out_physical_id')

    #Load the dictionary nested in the display dictionary
    raw_dict = json.loads(response.data).get('display')

    #Only keep the keys that are needed
    geo_dict = {k: raw_dict[k] for k in out_keys}

    return geo_dict

In [ ]:
def flat_list(in_list=None):
    
    #This function will take the return from Geosupport Function BN aka BIN and make the return all of the same type.
    #The raw return could produce a mixed list that contains a list of length 1 with 1 dictionary, a list of length 
    #1+n with 1+n dictionaries or simply a dictionary (if a GRC code is encountered).
    
    #Initialize the new list
    new_list = []
    
    #Iterate through the input list elements
    for i in in_list:
        #If the element is a list instance/type then check the length.
        if isinstance(i, list):
            #If the list contains more than one element (length > 1) then extract each element and append as a
            #type list element to the new list.
            if len(i) > 1:
                for j in i:
                    new_list.append([j])
            else:
            #Otherwise append the single type list element to the new list.
                new_list.append(i)
        #If the element is not a list instance/type (it will be a dict) then append as a type list element to the new
        #list.
        else:
            new_list.append([i])
    #new_list is now a list of lists with each inner list containing a single dictionary.
    
    #Flatten the list of list so that the return is a list of dictionaries and the inner list is removed.
    #Extract all inner list elements (dictionaries) from the outer list element.
    return_list = [inner_el for outer_el in new_list for inner_el in outer_el]
    
    #Return the flattened list.
    return return_list

In [ ]:
#Define a function that strips the leading and trailing spaces from the returned values of the dictionaries
def strip_vals(in_list):
    for dicts in in_list:
        dicts.update((k, v.strip()) for k, v in dicts.items() if isinstance(v, str) )
        #return dicts

In [ ]:
def replace_na(in_list):
    for dicts in in_list:
        dicts.update((k, None) for k, v in dicts.items() if v == 'N/A' or (isinstance(v, str) and v == ''))
        #return dicts

In [ ]:
def add_ck(in_list):
    for dicts in in_list:
        if dicts['out_grc'].strip('00') == '':
            
            #Check for equality and validity of low and high house number. The low and high house numbers need cannot be blank and
            #must be equal to use as input into function 1.
            if (dicts['high_address_number'].strip() == dicts['low_address_number'].strip() and 
               (dicts['high_address_number'].strip() != '' and dicts['low_address_number'].strip() != '')):          
                add_val = {'addressable': 'Addressable'}
                
            #If the low and high range are not equal then the record should not be input into function 1.
            else:
                add_val = {'addressable': 'Non-Addressable, Range'}
                
        
        else:
            add_val = {'addressable':'Non-Addressable: {}'.format(dicts['out_error_message'].strip())}
            
        dicts.update(add_val)
    #return dicts

## Call the functions

In [ ]:
def master_func(in_bins):
    list_of_things = []

    for bn in in_bins:
        list_of_things.append(funcbn(bn, out_keys = out_keys, grc_err = grc_err, api_key = geo_key, ip = geo_ip))
    
    t = flat_list(list_of_things)
    
    strip_vals(t)
    add_ck(t)
    
    for dicts in t:
        if dicts['addressable'] == 'Addressable':
            new_dict = func1b(dicts['out_sanborn_boro'], dicts['high_address_number'], dicts['street_name'], geo_key, geo_ip)
            dicts.update(new_dict)
    
    strip_vals(t)        
    replace_na(t)
    
    return_df = pd.DataFrame(t)
    return return_df

In [ ]:
df = master_func(bins)

In [ ]:
#Rename all of the columns from Geosupport so that they map exactly to the schema in SQL Server
dff = (df.rename(columns = {'out_zip_code': 'Zip_Code',
                               'b7sc': 'B7SC',
                               'out_b10sc1': 'B10SC',
                               'out_sanborn_boro': 'Boro_Code',
                               'bin': 'BIN',
                               'out_preferred_lgc': 'LGC',
                               'type_meaning': 'Address_Type',
                               'high_address_number': 'High_House_Num',
                               'low_address_number': 'Low_House_Num',
                               'out_sos_ind': 'Street_Side',
                               'street_name': 'Norm_Street_Name',
                               'out_usps_city_name': 'USPS_City',
                               'out_TPAD_bin_status': 'TPAD_BIN_Status',
                               'out_co': 'City_Council',
                               'out_ad': 'NYS_Assembly',
                               'out_sd': 'NYS_Senate',
                               'out_cd': 'US_Congress',
                               'out_nta': 'NTA_Code',
                               'out_fire_bat': 'Fire_Battalion',
                               'out_fire_co': 'Fire_Co_Num',
                               'out_fire_co_str': 'Fire_Co_Type',
                               'out_fire_div': 'Fire_Division',
                               'out_hurricane_zone': 'HEZ',
                               'out_police_patrol_boro': 'Police_Boro',
                               'Police Patrol Borough Command': 'Police_Boro_Com',
                               'out_police_pct': 'Police_Precinct',
                               'Sanitation Collection Scheduling Section and Subsection': 'Sanitation_Subsect',
                               'Sanitation District': 'Sanitation_District',
                               'Sanitation Recycling Collection Schedule': 'Sanitation_Recycling',
                               'out_san_reg': 'Sanitation_Reg_Pickup',
                               'out_physical_id': 'Physical_ID'})
       .reindex(columns = ['BIN', 'Boro_Code', 'Address_Type', 'Low_House_Num', 'High_House_Num', 'Norm_Street_Name', 'USPS_City', 
                           'Zip_Code', 'Physical_ID', 'B7SC', 'B10SC', 'LGC', 'Street_Side', 'TPAD_BIN_Status', 'HEZ', 
                           'Community_Board', 'City_Council', 'NYS_Assembly', 'NYS_Senate', 'US_Congress', 'NTA_Code', 
                           'Fire_Battalion', 'Fire_Co_Num', 'Fire_Co_Type', 'Fire_Division', 
                           'Police_Boro', 'Police_Boro_Com', 'Police_Precinct', 
                           'Sanitation_Subsect', 'Sanitation_District', 'Sanitation_Recycling', 'Sanitation_Reg_Pickup'])
       .drop_duplicates())

In [ ]:
measurer = np.vectorize(len)

In [ ]:
type_df = pd.DataFrame({'df_name': dff.columns.tolist(),
                        'df_type': dff.dtypes.astype(str).tolist(),
                        'df_len' : measurer(dff.values.astype(str)).max(axis=0)})

In [ ]:
type_df.head()

In [ ]:
len(dff)

In [ ]:
dff.head()

In [ ]:
con_str = "Driver={SQL Server};Server=" + vpipm_server + ";Database=structuresdb;Trusted_Connection=Yes;"
sa_con = quote_plus(con_str)
engine = sqlalchemy.create_engine(
    "mssql+pyodbc:///?odbc_connect={}".format(sa_con))

In [ ]:
if len(dff) > 0:
    dff.to_sql('tbl_geosupport_address', engine, schema = 'dbo', if_exists='replace', index=False)

In [ ]:
# dff.fillna(-9999, inplace=True)

In [ ]:
dff.Boro_Code.fillna(0,inplace=True)

In [ ]:
geosupport_FSET = arcgis.features.FeatureSet.from_dataframe(dff)

In [ ]:
# geosupport_FSET.features

In [ ]:
if len(dff) > 0:
    tbl_geosupport.delete_features(where="objectid > 0")

In [ ]:
if len(dff) >0 :
    add_to_geosupport = tbl_geosupport.edit_features(adds = geosupport_FSET.features)